In [1]:
# Advent of Code 2023
# Day 18 Problem 1
import re
from collections import Counter
import functools

with open("aoc_18_input.txt") as f:
    A = f.read().strip().split("\n")

# c+p test case here
TEST = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)""".strip().split("\n")

for i, line in enumerate(TEST):
    print(f"{i}:\t{line}")

0:	R 6 (#70c710)
1:	D 5 (#0dc571)
2:	L 2 (#5713f0)
3:	D 2 (#d2c081)
4:	R 2 (#59c680)
5:	D 2 (#411b91)
6:	L 5 (#8ceee2)
7:	U 2 (#caa173)
8:	L 1 (#1b58a2)
9:	U 2 (#caa171)
10:	R 2 (#7807d2)
11:	U 3 (#a77fa3)
12:	L 2 (#015232)
13:	U 2 (#7a21e3)


In [2]:
def parse(inp):
    res = []
    for row in inp:
        a,b,c = row.split(" ")
        res.append([a, int(b)])
    x, y = 0, 0
    minx, maxx, miny, maxy = 999_999_999, 0, 999_999_999, 0
    for dir, quant in res:
        if dir == "R":
            y += quant
        elif dir == "L":
            y -= quant
        elif dir == "U":
            x -= quant
        elif dir == "D":
            x += quant

        miny = min(miny, y)
        maxy = max(maxy, y)
        minx = min(minx, x)
        maxx = max(maxx, x)
    return res, (minx, maxx, miny, maxy)

def printMap(M):
    for row in M:
        print("".join(row))
    print()

# core of this is from day10 part2
def floodOutside(tap, start):
    queue = [[start[0],start[1]]]
    H, W = len(tap)-1, len(tap[0])-1
    while queue:
        nx,ny = queue.pop()
        tap[nx] = tap[nx][:ny] + [" "] + tap[nx][ny+1:]
        if nx > 0 and tap[nx-1][ny] == ".":     queue.append([nx-1,ny])
        if ny > 0 and tap[nx][ny-1] == ".":     queue.append([nx,ny-1])
        if nx < H and tap[nx+1][ny] == ".":     queue.append([nx+1,ny])
        if ny < W and tap[nx][ny+1] == ".":     queue.append([nx,ny+1])
    return tap
    
def buildMap(directions, height, width, start):
    M = [["." for _ in range(width)] for _ in range(height)]
    currX, currY = start
    M[currX][currY] = "#"
    dd = {
        "R": (0,1), 
        "L": (0,-1),
        "U": (-1,0),
        "D": (1,0)
        }
    
    for dir, length in directions:
        dx, dy = dd[dir]
        for _ in range(length):
            currX += dx
            currY += dy
            M[currX][currY] = "#"
    # printMap(M)
    return M

def countInside(M):
    res = 0
    for row in M:
        res += row.count(".")
        res += row.count("#")
    return res

def p1(inp):
    inst, dims = parse(inp)
    (minx, maxx, miny, maxy) = dims
    maxWidth = maxy - miny + 3
    maxHeight = maxx - minx + 3
    # print(f"X: {minx} - {maxx}, so height of {maxHeight}")
    # print(f"Y: {miny} - {maxy}, so width of {maxWidth}")
    start = (abs(minx)+1, abs(miny)+1)
    mapWithBorder = buildMap(inst, maxHeight, maxWidth, start)
    emptyOutside = floodOutside(mapWithBorder, (0,0))
    # printMap(emptyOutside)
    return countInside(emptyOutside)
    

In [3]:
# expected output: 62
p1(TEST)

62

In [4]:
p1(A)

46334

In [5]:
def parseP2(inp):
    res = []
    dir = {
        "0": "R", 
        "1": "D", 
        "2": "L", 
        "3": "U"
    }
    for row in inp:
        a,b,c = row.split(" ")
        hexcode = c[2:-1]
        inst = dir[hexcode[-1]]
        quant = int(hexcode[:-1], 16)
        res.append([inst, quant])
    x, y = 0, 0
    minx, maxx, miny, maxy = 999_999_999, 0, 999_999_999, 0
    for dir, quant in res:
        if dir == "R":
            y += quant
        elif dir == "L":
            y -= quant
        elif dir == "U":
            x -= quant
        elif dir == "D":
            x += quant

        miny = min(miny, y)
        maxy = max(maxy, y)
        minx = min(minx, x)
        maxx = max(maxx, x)
    return res, (minx, maxx, miny, maxy)

def p2(inp):
    # let's try a naive approach I guess?
    inst, dims = parseP2(inp)
    (minx, maxx, miny, maxy) = dims
    maxWidth = maxy - miny + 3
    maxHeight = maxx - minx + 3
    print(f"X: {minx} - {maxx}, so height of {maxHeight}")
    print(f"Y: {miny} - {maxy}, so width of {maxWidth}")
    start = (abs(minx)+1, abs(miny)+1)
    mapWithBorder = buildMap(inst, maxHeight, maxWidth, start)
    print(f"Done with buildMap()")
    emptyOutside = floodOutside(mapWithBorder, (0,0))
    # printMap(emptyOutside)
    return countInside(emptyOutside)

In [6]:
# expected output: 952408144115
# this currently doesn't halt in a minute
#p2(TEST)

In [7]:
%%capture
"""

This is the first day where I had to look up a completely different approach. So I'm reading about the Shoelace formula and Pick's Theorem and learning about those.

PICK'S THEOREM:
---------------
Suppose that a polygon has integer coordinates for all of its vertices.
Let i be the number of integer points interior to the polygon, and 
let b be the number of integer points on its boundary (including both vertices and points along the sides). 
Then the area A of this polygon is:

    A = i + b/2 - 1 

SHOELACE FORMULA
----------------

        | x_1 x_2 x_3       x_n x_1 |   | x_1 x_2 |   | x_2 x_3 |       | x_n x_1 |   
    2A = |             . . .         | = |         | + |         | + ... |         |
        | y_1 y_2 y_3       y_n y_1 |   | y_1 y_2 |   | x_2 x_3 |       | x_n x_1 |

    2A = (x_1 * y_2 - x_2 * y_1) + (x_2 * y_3 - x_3 * y_2) + ... (x_n * y_1 - x_1 * y_n)   

(when going counterclockwise, since we are going clockwise [inspecting input] we need to take abs() of this)

We are looking for the number of points in the boundary and interior, so i = A - b/2 + 1
and we can actually just count the boundary points by summing the lengths while parsing input
so altogether:

    i = abs( 1/2 * [(x_1 * y_2 - x_2 * y_1) + (x_2 * y_3 - x_3 * y_2) + ... (x_n * y_1 - x_1 * y_n)] ) - b/2 + 1

"""

In [8]:
def parseP2GetCoords(inp):
    res = []
    dir = {
        "0": "R", 
        "1": "D", 
        "2": "L", 
        "3": "U"
    }
    for row in inp:
        _,_,color = row.split(" ")
        hexcode = color[2:-1]
        inst = dir[hexcode[-1]]
        quant = int(hexcode[:-1], 16)
        res.append([inst, quant])
    x, y = 0, 0
    minx, maxx, miny, maxy = 999_999_999, 0, 999_999_999, 0
    for dir, quant in res:
        if dir == "R":      y += quant
        elif dir == "L":    y -= quant
        elif dir == "U":    x -= quant
        elif dir == "D":    x += quant

        miny = min(miny, y)
        maxy = max(maxy, y)
        minx = min(minx, x)
        maxx = max(maxx, x)
    return res, (minx, maxx, miny, maxy)

def getCoords(inst, dims, start):
    sx, sy = start
    boundaryPoints = 0
    res = [(start)]
    unit = {
        "R": (0,1), 
        "D": (1,0), 
        "L": (0,-1), 
        "U": (-1,0)
    }
    for dir, magnitude in inst:
        boundaryPoints += magnitude
        prev = res[-1]
        dx, dy = unit[dir]
        xx = prev[0] + magnitude * dx
        yy = prev[1] + magnitude * dy
        res.append((xx,yy))
    return boundaryPoints, res
    
def picksShoelace(b, points):
    # shoelace formula
    tmp = 0
    points.append(points[0])
    for i, (x,y) in enumerate(points[:-1]):
        xx, yy = points[i+1]
        tmp += x * yy - y * xx
    A = abs(tmp) // 2

    # picks thm
    i = A -( (b) // 2) +1
    return i + b

def p2attempt2(inp):
    # Let's try Shoelace Formula + Pick's Thm
    inst, dims = parseP2GetCoords(inp)
    (minx, maxx, miny, maxy) = dims
    maxWidth = maxy - miny + 3
    maxHeight = maxx - minx + 3
    # print(f"X: {minx} - {maxx}, so height of {maxHeight}")
    # print(f"Y: {miny} - {maxy}, so width of {maxWidth}")
    start = (abs(minx)+1, abs(miny)+1)
    boundaryCount, vertices = getCoords(inst, dims, start)
    return picksShoelace(boundaryCount, vertices)



In [9]:
# expected output: 952408144115
p2attempt2(TEST)

952408144115

In [10]:
p2attempt2(A)

102000662718092